# Assignment: Data Wrangling
### `! git clone https://github.com/ds4e/wrangling`
### Do Q1 and Q2, and either Q3 or Q4, for a total of 3 questions.

In [4]:
! git clone https://github.com/ds4e/wrangling

Cloning into 'wrangling'...
remote: Enumerating objects: 63, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 63 (delta 2), reused 1 (delta 0), pack-reused 56 (from 1)
Receiving objects: 100% (63/63), 13.96 MiB | 22.73 MiB/s, done.
Resolving deltas: 100% (12/12), done.


**Q1.** This question provides some practice cleaning variables which have common problems.
1. Numeric variable: For `./data/airbnb_hw.csv`, clean the `Price` variable as well as you can, and explain the choices you make. How many missing values do you end up with? (Hint: What happens to the formatting when a price goes over 999 dollars, say from 675 to 1,112?)
2. Categorical variable: For the Minnesota police use of for data, `./data/mn_police_use_of_force.csv`, clean the `subject_injury` variable, handling the NA's; this gives a value `Yes` when a person was injured by police, and `No` when no injury occurred. What proportion of the values are missing? Is this a concern? Cross-tabulate your cleaned `subject_injury` variable with the `force_type` variable. Are there any patterns regarding when the data are missing?
3. Dummy variable: For the pretrial data covered in the lecture, clean the `WhetherDefendantWasReleasedPretrial` variable as well as you can, and, in particular, replace missing values with `np.nan`.
4. Missing values, not at random: For the pretrial data covered in the lecture, clean the `ImposedSentenceAllChargeInContactEvent` variable as well as you can, and explain the choices you make. (Hint: Look at the `SentenceTypeAllChargesAtConvictionInContactEvent` variable.)

In [5]:
#1
import pandas as pd
import numpy as np
df = pd.read_csv('/content/wrangling/assignment/data/airbnb_hw.csv')

#Remove non-numeric characters
df['Price'] = df['Price'].str.replace(r'[^\d.]', '', regex=True)

#Convert to numeric
df['Price'] = pd.to_numeric(df['Price'], errors='coerce')

#Handle missing values
missing_price_count = df['Price'].isna().sum()
print(f"Number of missing values in Price after cleaning: {missing_price_count}")

Number of missing values in Price after cleaning: 0


In [7]:
#2
police_data = pd.read_csv('/content/wrangling/assignment/data/mn_police_use_of_force.csv')

#Clean subject_injury variable
police_data['subject_injury'] = police_data['subject_injury'].fillna('Unknown')
police_data['subject_injury'] = police_data['subject_injury'].map({'Yes': 'Yes', 'No': 'No', 'Unknown': np.nan})

#Calculate proportion of missing values
missing_injury_proportion = police_data['subject_injury'].isna().mean()
print(f"Proportion of missing values in subject_injury: {missing_injury_proportion:.2%}")

#Cross-tabulate with force_type
crosstab = pd.crosstab(police_data['subject_injury'], police_data['force_type'])
print(crosstab)


Proportion of missing values in subject_injury: 76.19%
force_type      Baton  Bodily Force  Chemical Irritant  Firearm  \
subject_injury                                                    
No                  0          1093                131        2   
Yes                 2          1286                 41        0   

force_type      Gun Point Display  Improvised Weapon  Less Lethal Projectile  \
subject_injury                                                                 
No                             33                 34                       1   
Yes                            44                 40                       2   

force_type      Police K9 Bite  Taser  
subject_injury                         
No                           2    150  
Yes                         44    172  


In [10]:
#3
pretrial_data = pd.read_parquet('/content/wrangling/assignment/data/justice_data.parquet')

#Clean WhetherDefendantWasReleasedPretrial variable
pretrial_data['WhetherDefendantWasReleasedPretrial'] = pretrial_data['WhetherDefendantWasReleasedPretrial'].map({'Yes': 1, 'No': 0})
pretrial_data['WhetherDefendantWasReleasedPretrial'] = pretrial_data['WhetherDefendantWasReleasedPretrial'].replace({np.nan: np.nan})

#Check cleaned data
print(pretrial_data['WhetherDefendantWasReleasedPretrial'].head())


0   NaN
1   NaN
2   NaN
3   NaN
4   NaN
Name: WhetherDefendantWasReleasedPretrial, dtype: float64


In [11]:
#4
#Clean ImposedSentenceAllChargeInContactEvent
pretrial_data['ImposedSentenceAllChargeInContactEvent'] = pretrial_data['ImposedSentenceAllChargeInContactEvent'].fillna('No Sentence')  # Example handling

#Investigate relationship with SentenceTypeAllChargesAtConvictionInContactEvent
correlation_check = pretrial_data.groupby('SentenceTypeAllChargesAtConvictionInContactEvent')['ImposedSentenceAllChargeInContactEvent'].apply(lambda x: x.isna().mean())
print(correlation_check)


SentenceTypeAllChargesAtConvictionInContactEvent
0    0.0
1    0.0
2    0.0
4    0.0
9    0.0
Name: ImposedSentenceAllChargeInContactEvent, dtype: float64


**Q2.** Go to https://sharkattackfile.net/ and download their dataset on shark attacks.

1. Open the shark attack file using Pandas. It is probably not a csv file, so `read_csv` won't work.
2. Drop any columns that do not contain data.
3. Clean the year variable. Describe the range of values you see. Filter the rows to focus on attacks since 1940. Are attacks increasing, decreasing, or remaining constant over time?
4. Clean the Age variable and make a histogram of the ages of the victims.
5. What proportion of victims are male?
6. Clean the `Type` variable so it only takes three values: Provoked and Unprovoked and Unknown. What proportion of attacks are unprovoked?
7. Clean the `Fatal Y/N` variable so it only takes three values: Y, N, and Unknown.
8. Are sharks more likely to launch unprovoked attacks on men or women? Is the attack more or less likely to be fatal when the attack is provoked or unprovoked? Is it more or less likely to be fatal when the victim is male or female? How do you feel about sharks?
9. What proportion of attacks appear to be by white sharks? (Hint: `str.split()` makes a vector of text values into a list of lists, split by spaces.)

**Q3.** Open the "tidy_data.pdf" document in the repo, which is a paper called Tidy Data by Hadley Wickham.

  1. Read the abstract. What is this paper about?
  2. Read the introduction. What is the "tidy data standard" intended to accomplish?
  3. Read the intro to section 2. What does this sentence mean: "Like families, tidy datasets are all alike but every messy dataset is messy in its own way." What does this sentence mean: "For a given dataset, it’s usually easy to figure out what are observations and what are variables, but it is surprisingly difficult to precisely define variables and observations in general."
  4. Read Section 2.2. How does Wickham define values, variables, and observations?
  5. How is "Tidy Data" defined in section 2.3?
  6. Read the intro to Section 3 and Section 3.1. What are the 5 most common problems with messy datasets? Why are the data in Table 4 messy? What is "melting" a dataset?
  7. Why, specifically, is table 11 messy but table 12 tidy and "molten"?
  8. Read Section 6. What is the "chicken-and-egg" problem with focusing on tidy data? What does Wickham hope happens in the future with further work on the subject of data wrangling?

**Q4.** Many important datasets contain a race variable, typically limited to a handful of values often including Black, White, Asian, Latino, and Indigenous. This question looks at data gathering efforts on this variable by the U.S. Federal government.

1. How did the most recent US Census gather data on race?
2. Why do we gather these data? What role do these kinds of data play in politics and society? Why does data quality matter?
3. Please provide a constructive criticism of how the Census was conducted: What was done well? What do you think was missing? How should future large scale surveys be adjusted to best reflect the diversity of the population? Could some of the Census' good practices be adopted more widely to gather richer and more useful data?
4. How did the Census gather data on sex and gender? Please provide a similar constructive criticism of their practices.
5. When it comes to cleaning data, what concerns do you have about protected characteristics like sex, gender, sexual identity, or race? What challenges can you imagine arising when there are missing values? What good or bad practices might people adopt, and why?
6. Suppose someone invented an algorithm to impute values for protected characteristics like race, gender, sex, or sexuality. What kinds of concerns would you have?